In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from src import *

In [4]:
import numpy as np
import random
import pandas as pd
import torch
from torch import nn
from collections import defaultdict
from tqdm.auto import tqdm, trange
import pickle as pkl
from typing import Literal, get_type_hints

import itertools as it
import more_itertools as mit

In [5]:
def player_formatter(player: int) -> str:
    symbol_mapping = {0: " ", 1: "X", -1: "O"}
    return symbol_mapping[player]
    
def state_formatter(state: tuple[int, ...]) -> str:
    size = int(len(state) ** 0.5)
    formatted_state = "\n"
    for i in range(size):
        formatted_state += "+---+---+---+\n"
        row = state[i*size:(i+1)*size]
        formatted_state += "| " + " | ".join(player_formatter(cell)for cell in row) + " |\n"
    formatted_state += "+---+---+---+"
    return formatted_state

In [6]:
ttt = TicTacToe(default_state_formatter=state_formatter)
ttt.render()


+---+---+---+
|   |   |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   |   |   |
+---+---+---+


In [7]:
q_table_policy = QTablePolicy.load('q_table', lr=0.1, name='Deterministic Q Table')
q_table_policy_stochastic = QTablePolicy(q_table_policy.q_table, 
                                         lr=0.1,
                                         stochastic=True,
                                         temperature=0.5,
                                         name='Stochastic Q Table')
human_policy = PromptPolicy(player_formatter)
min_max_policy = MinMaxPolicy(game_cls=TicTacToe)
mcts_policy = MCTS(game_cls=TicTacToe)
# mlp_policy = MLPPolicy(mlp=MLP(), lr=0.001, name='MLP Policy')
mlp_policy = MLPPolicy.load('mlp', mlp=MLP(), lr=0.001, device='cpu', name='MLP Policy')
# mcts_policy = MCTS(ttt)

In [8]:
def play(ttt, policy=None, print_state=False):
    if policy is None:
        policy = RandomPolicy()
    if not isinstance(policy, dict):
        policy = {-1: policy, 1: policy}

    tape = [dict(
        player=None,
        action_space=None,
        action=None,
        state=ttt.get_state(),
        winner=0
    )]

    while (action_space := ttt.get_actions()) and not ttt.get_winner():
        if print_state:
            ttt.render()
        player = ttt.player
        action = ttt.agent_move(policy[player])
        state = ttt.get_state()
        winner = ttt.get_winner()
        tape.append(dict(
            player=player,
            action_space=action_space,
            action=action,
            state=state,
            winner=winner
        ))
    if print_state:
        ttt.render()

    tape.append(dict(
        player=None,
        action_space=set(),
        action=None,
        state=ttt.get_state(),
        winner=ttt.get_winner()
    ))
    
    return tape

In [9]:
def swap(state: np.ndarray, option: Literal[1, -1]):
    return option * state

def flip(state: np.ndarray, option: Literal[True, False]):
    return np.fliplr(state) if option else state

def rotate(state: np.ndarray, option: Literal[0, 1, 2, 3]):
    return np.rot90(state, k=option)
    

In [10]:
def get_param_options(fn, param='option'):
    return get_type_hints(fn)['option'].__args__

In [11]:
def transform_state(raw_state, fns, opts):
    state = raw_state.reshape([3, 3])
    for fn, opt in zip(fns, opts):
        state = fn(state, opt)
    return state.flatten()

def transform_actions(raw_actions, fns, opts):
    actions = np.zeros(9)
    actions[raw_actions] = 1
    actions = transform_state(actions, fns, opts)
    return np.nonzero(actions)[0]

In [12]:
def bellman_equation(policy, reward, state, player, actions):
    if not actions:
        return reward
    return reward + max(policy.get_Q(state, player, action) for action in actions)

def replay_episode(tape, policy):
    state_list, player_list, action_list, q_list = [], [], [], []
    transformations = [swap, flip, rotate]

    for pre, cur, nxt in mit.windowed(tape, 3):
        raw_start_state = np.array(list(pre['state'])).astype(int)
        raw_end_state = np.array(list(nxt['state'])).astype(int)
        raw_action = np.array([cur['action']]).astype(int)
        raw_action_space = np.array(list(nxt['action_space'])).astype(int)
        player = cur['player']
        reward = cur['player'] * nxt['winner']
        
        for opts in it.product(*map(get_param_options, transformations)):
            start_state = tuple(transform_state(raw_start_state, transformations, opts).tolist())
            end_state = tuple(transform_state(raw_end_state, transformations, opts).tolist())
            action = transform_actions(raw_action, transformations, opts).item()
            action_space = set(transform_actions(raw_action_space, transformations, opts).tolist())

            new_q = bellman_equation(policy, reward, end_state, player, action_space)
            state_list.append(start_state)
            player_list.append(player)
            action_list.append(action)
            q_list.append(new_q)

    return state_list, player_list, action_list, q_list

In [12]:
def train(policy, episodes, epsilon=0.1):
    eps_greedy_policy = EpsilonGreedyPolicy(policy, epsilon=epsilon)
    for episode in trange(episodes):
        ttt.reset(start_player=random.choice([1, -1]))
        tape = play(ttt, policy=eps_greedy_policy)
        state_list, player_list, action_list, q_list = replay_episode(tape, policy)
        loss = policy.batch_update_Q(state_list, player_list, action_list, q_list)


In [13]:
train(mlp_policy, episodes=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
mlp_policy.save('mlp')

In [ ]:
train(q_table_policy_stochastic, episodes=100000)

In [ ]:
q_table_policy.save('q_table')

## Manual Play

In [13]:
ttt.reset()
tape = play(ttt, policy={1: human_policy, -1: q_table_policy}, print_state=True)


+---+---+---+
|   |   |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   |   |   |
+---+---+---+

+---+---+---+
|   |   |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   |   |   |
+---+---+---+

+---+---+---+
| O |   |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   |   |   |
+---+---+---+

+---+---+---+
| O |   |   |
+---+---+---+
|   | X | X |
+---+---+---+
|   |   |   |
+---+---+---+

+---+---+---+
| O |   |   |
+---+---+---+
| O | X | X |
+---+---+---+
|   |   |   |
+---+---+---+

+---+---+---+
| O |   |   |
+---+---+---+
| O | X | X |
+---+---+---+
| X |   |   |
+---+---+---+

+---+---+---+
| O |   | O |
+---+---+---+
| O | X | X |
+---+---+---+
| X |   |   |
+---+---+---+

+---+---+---+
| O | X | O |
+---+---+---+
| O | X | X |
+---+---+---+
| X |   |   |
+---+---+---+

+---+---+---+
| O | X | O |
+---+---+---+
| O | X | X |
+---+---+---+
| X | O |   |
+---+---+---+

+---+---+---+
| O | X | O |
+---+---+---+
| O | X | X |
+---+---+---+
| X | O | X |
+---+---+---+


In [14]:
pd.DataFrame(tape)

,player,action_space,action,state,winner
0,NaN,None,NaN,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",0
1,1.0,"{0, 1, 2, 3, 4, 5, 6, 7, 8}",4.0,"(0, 0, 0, 0, 1, 0, 0, 0, 0)",0
2,-1.0,"{0, 1, 2, 3, 5, 6, 7, 8}",0.0,"(-1, 0, 0, 0, 1, 0, 0, 0, 0)",0
3,1.0,"{1, 2, 3, 5, 6, 7, 8}",5.0,"(-1, 0, 0, 0, 1, 1, 0, 0, 0)",0
4,-1.0,"{1, 2, 3, 6, 7, 8}",3.0,"(-1, 0, 0, -1, 1, 1, 0, 0, 0)",0
5,1.0,"{1, 2, 6, 7, 8}",6.0,"(-1, 0, 0, -1, 1, 1, 1, 0, 0)",0
6,-1.0,"{8, 1, 2, 7}",2.0,"(-1, 0, -1, -1, 1, 1, 1, 0, 0)",0
7,1.0,"{8, 1, 7}",1.0,"(-1, 1, -1, -1, 1, 1, 1, 0, 0)",0
8,-1.0,"{8, 7}",7.0,"(-1, 1, -1, -1, 1, 1, 1, -1, 0)",0
9,1.0,{8},8.0,"(-1, 1, -1, -1, 1, 1, 1, -1, 1)",0


# Run Simulations

In [15]:
def simulate(round, player_policy_map, start_player=None):
    player_policy_map[0] = DummyPolicy(name='Tie')
    win_count = {policy.get_name(): 0 for policy in player_policy_map.values()}
    for episode in trange(round):
        ttt.reset(start_player=start_player or random.choice([-1, 1]))
        tape = play(ttt, policy=player_policy_map)
        win_count[player_policy_map[ttt.get_winner()].get_name()] += 1
    return win_count

In [16]:
policy_1 = RandomPolicy(name='Random Policy')
# policy_1 = q_table_policy
policy_2 = mlp_policy

result_list = []
for p1_role, start_player in [[1, 1], [1, -1], [-1, 1], [-1, -1]]:
    policy_map = {1*p1_role: policy_1, -1*p1_role: policy_2}
    result = simulate(100, policy_map, start_player=start_player)
    result['Player -1'] = policy_map[-1].get_name()
    result['Player 1'] = policy_map[1].get_name()
    result['Start Player'] = policy_map.get(start_player).get_name()
    result_list.append(result)
pd.DataFrame(result_list)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

,Random Policy,MLP Policy,Tie,Player -1,Player 1,Start Player
0,16,73,11,MLP Policy,Random Policy,Random Policy
1,2,93,5,MLP Policy,Random Policy,MLP Policy
2,1,95,4,Random Policy,MLP Policy,MLP Policy
3,11,81,8,Random Policy,MLP Policy,Random Policy
